In [2]:
from langchain.vectorstores import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# from langchain.document_loaders import TextLoader
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import DirectoryLoader


# from InstructorEmbedding import INSTRUCTOR
# from langchain.embeddings import HuggingFaceInstructEmbeddings

In [3]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
persist_directory = 'db_500_100'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

In [5]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [1]:
!nvidia-smi

Thu Aug  3 15:59:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   63C    P0              31W /  70W |      2MiB / 15360MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
q1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"
ans1 = retriever.get_relevant_documents(q1)

In [7]:
q2 = "What does the Reminder field indicate when submitting an RMA request?"
ans2 = retriever.get_relevant_documents(q2)

In [8]:
q3 = "What information should be filled in the Customer No. field when submitting an RMA request?"
ans3 = retriever.get_relevant_documents(q3)

In [9]:
q4 = "What steps should be followed when filling in the request information for an RMA?"
ans4 = retriever.get_relevant_documents(q4)

In [10]:
q5 = "Can this document be used for purposes other than introducing Wareconn functions and procedures?"
ans5 = retriever.get_relevant_documents(q5)

In [11]:
llm_name = "ggml-gpt4all-j-v1.3-groovy.bin"

In [12]:
import gpt4all
gptj = gpt4all.GPT4All(llm_name,allow_download=False, model_path='/root/.cache/gpt4all/')

Found model file at  /root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from '/root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [20]:
# llm_name = "ggml-model-gpt4all-falcon-q4_0.bin"
# falcon = gpt4all.GPT4All(llm_name,allow_download=False, model_path='/root/.cache/gpt4all/')

In [21]:
!ls -lah /root/.cache/gpt4all/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 7.4G
drwxr-xr-x  2 root root   98 2023-08-02__16:17:05 .
drwxr-xr-x. 8 root root   91 2023-07-31__14:13:10 ..
-rw-r--r--  1 root root 3.6G 2023-05-10__20:09:23 ggml-gpt4all-j-v1.3-groovy.bin
-rw-r--r--  1 root root 3.8G 2023-06-28__00:32:51 ggml-model-gpt4all-falcon-q4_0.bin
-rw-r--r--  1 root root  157 2023-08-02__16:17:05 p.py


In [15]:
!ls -lah ./models

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 3.8G
drwxr-xr-x 2 root root   56 2023-08-02__11:26:33 .
drwxrwxrwx 6 root root 4.0K 2023-08-03__15:45:15 ..
-rw-r--r-- 1 root root 3.8G 2023-08-02__15:40:08 ggml-model-gpt4all-falcon-q4_0.bin


In [16]:
import inspect

In [17]:
inspect.signature(gptj.generate)

<Signature (prompt: str, max_tokens: int = 200, temp: float = 0.7, top_k: int = 40, top_p: float = 0.4, repeat_penalty: float = 1.18, repeat_last_n: int = 64, n_batch: int = 8, n_predict: Optional[int] = None, streaming: bool = False, callback: Callable[[int, str], bool] = <function empty_response_callback at 0x7f5acac768c0>) -> Union[str, Iterable[str]]>

In [18]:
def generate_from_template(context, query):
  return  f"""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  {context}

  Question: {query}
  Answer:"""

In [19]:

content_with_ans1 = ""
for i, a in enumerate(ans1):
  content_with_ans1+=a.page_content

prompt1 = generate_from_template(content_with_ans1, q1)
print(prompt1)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure  
2. Go to  RMA Request  Page 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims  
⚫ Description: Go to RMA request page  
 
To go to the RMA request page, follow these steps:  
Click Warranty Claims  in the left menu, then click +Add

  Question: How c

In [22]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res = gptj.generate(prompt1)
print(res)

 To access the RM
CPU times: user 2min 44s, sys: 1.15 s, total: 2min 46s
Wall time: 41.7 s


In [23]:
from rouge import Rouge
rouge_scorer = Rouge()

hypothesis = res
reference = "To navigate to the RMA request page, click on \"Warranty Claims\" in the left menu, then click on the \"+Add\" button."

score = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
score[0]["rouge-l"]["f"]

0.19047618739229027

In [24]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU(effective_order=True)

score = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference]
)

score.score/100 # sacreBLEU gives the score in percent

0.00036670698555488354

In [25]:
import inspect
inspect.signature(BLEU)

<Signature (lowercase: bool = False, force: bool = False, tokenize: Optional[str] = None, smooth_method: str = 'exp', smooth_value: Optional[float] = None, max_ngram_order: int = 4, effective_order: bool = False, trg_lang: str = '', references: Optional[Sequence[Sequence[str]]] = None)>

In [26]:
content_with_ans2 = ""
for i, a in enumerate(ans2):
  content_with_ans2+=a.page_content

prompt2 = generate_from_template(content_with_ans2, q2)
print(prompt2)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process6 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  
5. Submit the RMA  request  
 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit  
⚫ Description: Submit the RMA request  
 
The Reminder field will show you whether this RMA request is approvable

  Question: What does the Reminder field indicate when submitting an RMA request?
  Answer:


In [27]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res2 = gptj.generate(prompt2)

CPU times: user 2min 27s, sys: 683 ms, total: 2min 27s
Wall time: 37.1 s


In [28]:
print(res2)

 The Remiender field indicates that the RMAs are acceptable for warranty providers.


In [40]:
rouge_scorer = Rouge()

hypothesis = res2
reference = "The Reminder field shows whether the RMA request is approvable (acceptable) for the warranty provider or not."

rouge = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
print(rouge[0]["rouge-l"]["f"])

bleu_scorer = BLEU(effective_order=True)

blue = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference]
)

print(blue.score/100) # sacreBLEU gives the score in percent

0.357142852244898
0.02932576302133846


In [30]:
content_with_ans3 = ""
for i, a in enumerate(ans3):
  content_with_ans3+=a.page_content

prompt3 = generate_from_template(content_with_ans3, q3)
print(prompt3)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal

  Question: What information should be filled in the Customer No. field when submitting an RMA request?
  Answer:


In [31]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res3 = gptj.generate(prompt3)
print(res3)

 The customer number (WAL#) is a unique identifier for each individual who has purchased or received products from your company and needs to receive warranty service under their respective warranties. It can also serve as proof of purchase, which may be required by some third-party providers such as the manufacturer's authorized repair center.
CPU times: user 3min 3s, sys: 686 ms, total: 3min 3s
Wall time: 46.2 s


In [41]:
rouge_scorer = Rouge()

hypothesis = res3
reference = "When submitting an RMA request, you should fill in the WAL# (Warranty Authorization Number) in the Customer No. field."

rouge = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
print(rouge[0]["rouge-l"]["f"])

bleu_scorer = BLEU(effective_order=True)

blue = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference]
)

print(blue.score/100) # sacreBLEU gives the score in percent

0.028985503532871725
0.017394593259606017


In [32]:
content_with_ans4 = ""
for i, a in enumerate(ans4):
  content_with_ans4+=a.page_content

prompt4 = generate_from_template(content_with_ans4, q4)
print(prompt4)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal

  Question: What steps should be followed when filling in the request information for an RMA?
  Answer:


In [33]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res4 = gptj.generate(prompt4)
print(res4)

 When filling out a warranty claim or return item authorization (RMA) form online through Wareconn's website, follow these general guidelines to ensure accurate and complete data entry. Here are some key points to keep in mind while completing this process:
1. Enter your customer number correctly - Make sure you enter the correct Customer No., which is a unique 10-digit code assigned by Wareconn for each of its customers. If you have multiple accounts, make sure you select the right one and provide all necessary information such as contact details (name, address), email addresses, phone numbers, etc.
2. Enter your RMA number correctly - Make sure to enter a unique 10-digit code assigned by Wareconn for each RMA request. If you have multiple requests in progress at any given time, make sure the correct one is selected and provide all necessary information such as contact details (name, address), email addresses, phone numbers, etc.
3. Enter your warranty provider's name correctly
CPU ti

In [42]:
rouge_scorer = Rouge()

hypothesis = res4
reference = """To fill in the request information for an RMA, follow these steps:

Fill in the Product SN (Serial Number), Product PN (Part Number), and defective reason.
Select the Rack PN (Part Number) by clicking on "Please select Parts" and choose the desired quantity.
Click on "Confirm" to proceed.
Optionally, fill in the Parts SN (Serial Number) for the failure part (not required).
Click on "Save" to save the information.
"""

rouge = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
print(rouge[0]["rouge-l"]["f"])

bleu_scorer = BLEU(effective_order=True)

blue = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference]
)

print(blue.score/100) # sacreBLEU gives the score in percent

0.13245032683829672
0.015184028482223562


In [34]:
content_with_ans5 = ""
for i, a in enumerate(ans5):
  content_with_ans5+=a.page_content

prompt5 = generate_from_template(content_with_ans5, q5)
print(prompt5)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  
 
 
 
 
 
 
wareconn standard operating procedure  
RMA request  SOP  
 
Version : 
Version  Date  Editor  Description  
v 1.0  2023/03/16 Eric Sun First draft (English version)  
 
 
Content5 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  5 wareconn standard operating procedure  
4. Fill in request info  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Fill in info  
⚫ Description:  
Fill in Product SN, Product PN

In [35]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res5 = gptj.generate(prompt5)
print(res5)

 No. This document is only intended to introduce Wareconn's functions and procedures related to warranty claims processing. It should not be used or relied upon in any way outside of the context provided, as it may contain proprietary information that could cause harm if misused.
CPU times: user 3min 41s, sys: 459 ms, total: 3min 42s
Wall time: 55.8 s


In [43]:
rouge_scorer = Rouge()

hypothesis = res5
reference = "No, this document explicitly states that it should not be used for other purposes. Its sole intention is to introduce Wareconn functions and procedures."

rouge = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
print(rouge[0]["rouge-l"]["f"])

bleu_scorer = BLEU(effective_order=True)

blue = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference]
)

print(blue.score/100) # sacreBLEU gives the score in percent

0.3235294071972319
0.08807590307622684
